
# Chain-of-Thought Variants

We compare three CoT approaches:

1. **Few-shot CoT (Human-crafted examples):** Supply solved examples with reasoning.
2. **Zero-shot CoT:** Add *"Let's think step by step"* to the prompt.
3. **Auto-CoT:** Ask the LLM to generate reasoning examples for similar questions, then use them as few-shot CoT.

> Based on [LearnPrompting: Automatic Chain of Thought](https://learnprompting.org/docs/advanced/thought_generation/automatic_chain_of_thought)


In [1]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


## 1. Few-shot CoT (human-crafted examples)

In [3]:
example_prompt = PromptTemplate.from_template(
    """Question: {q}
Reasoning: {r}
Final answer: {a}
"""
)

examples = [
    {
        "q": "If you have 2 apples and buy 3 more, how many total?",
        "r": "Start with 2, add 3, gives 5.",
        "a": "5",
    },
    {
        "q": "Tom has 5 pens, gives away 2, then buys 1 more. How many now?",
        "r": "5-2=3, plus 1 = 4.",
        "a": "4",
    },
]

fs_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {user_q}\nReasoning:",
    input_variables=["user_q"],
)


def few_shot_cot(q: str):
    return (fs_template | llm).invoke({"user_q": q}).content

In [4]:
print(fs_template.invoke({"user_q": "test"}).text)

Question: If you have 2 apples and buy 3 more, how many total?
Reasoning: Start with 2, add 3, gives 5.
Final answer: 5


Question: Tom has 5 pens, gives away 2, then buys 1 more. How many now?
Reasoning: 5-2=3, plus 1 = 4.
Final answer: 4


Question: test
Reasoning:


In [5]:

q1 = "Alice had 4 oranges, bought 2 more, then ate 1. How many left?"
print(few_shot_cot(q1))


Start with 4, add 2 to get 6, then subtract 1 for the orange eaten, which gives 5.  
Final answer: 5


## 2. Zero-shot CoT

In [9]:

zs_cot_tpl = PromptTemplate.from_template(
    """Question: {q}

Let's think step by step."""
)

def zero_shot_cot(q: str):
    return (zs_cot_tpl | llm).invoke({"q": q}).content

q2 = "A train moves at 50 km/h for 2 hours. How far does it travel?"
print(zero_shot_cot(q2))


To find out how far the train travels, we can use the formula for distance:

\[
\text{Distance} = \text{Speed} \times \text{Time}
\]

1. **Identify the speed of the train**: The train moves at a speed of 50 km/h.
2. **Identify the time the train travels**: The train travels for 2 hours.
3. **Plug the values into the formula**:

\[
\text{Distance} = 50 \, \text{km/h} \times 2 \, \text{hours}
\]

4. **Calculate the distance**:

\[
\text{Distance} = 100 \, \text{km}
\]

So, the train travels **100 kilometers**.


## 3. Auto-CoT (LLM generates examples, then apply)

In [10]:

# Step 1: Generate reasoning examples automatically for similar tasks
gen_tpl = PromptTemplate.from_template(
    """Generate 3 examples of math word problems with reasoning and final answer.

Format:
[{{
    "q": "...",
    "r": "...",
    "a": "..."
}},
...]""")

generated = (gen_tpl | llm).invoke({}).content
print("Generated Examples:\n", generated)


Generated Examples:
 [
    {
        "q": "A farmer has 120 apples. He wants to pack them into boxes, with each box holding 8 apples. How many boxes can he fill completely?",
        "r": "To find out how many boxes the farmer can fill, we divide the total number of apples by the number of apples per box. So, 120 apples ÷ 8 apples/box = 15 boxes.",
        "a": "15"
    },
    {
        "q": "A school has 250 students. If 60% of the students are girls, how many boys are there in the school?",
        "r": "First, we calculate the number of girls by finding 60% of 250. This is 0.60 × 250 = 150 girls. To find the number of boys, we subtract the number of girls from the total number of students: 250 - 150 = 100 boys.",
        "a": "100"
    },
    {
        "q": "A car travels 60 miles per hour. How far will it travel in 3.5 hours?",
        "r": "To find the distance traveled, we use the formula: distance = speed × time. Here, the speed is 60 miles/hour and the time is 3.5 hours. So, di

In [15]:
# Step 2: Use generated examples as few-shot CoT for a new question

auto_examples = [
    {
        "q": "A farmer has 120 apples. He wants to pack them into boxes, with each box holding 8 apples. How many boxes can he fill completely?",
        "r": "To find out how many boxes the farmer can fill, we divide the total number of apples by the number of apples per box. So, 120 apples ÷ 8 apples/box = 15 boxes.",
        "a": "15"
    },
    {
        "q": "A school has 250 students. If 60% of the students are girls, how many boys are there in the school?",
        "r": "First, we calculate the number of girls by finding 60% of 250. This is 0.60 × 250 = 150 girls. To find the number of boys, we subtract the number of girls from the total number of students: 250 - 150 = 100 boys.",
        "a": "100"
    },
    {
        "q": "A car travels 60 miles per hour. How far will it travel in 3.5 hours?",
        "r": "To find the distance traveled, we use the formula: distance = speed × time. Here, the speed is 60 miles/hour and the time is 3.5 hours. So, distance = 60 × 3.5 = 210 miles.",
        "a": "210"
    }
]

auto_template = FewShotPromptTemplate(
    examples=auto_examples,
    example_prompt=example_prompt,
    suffix="Question: {user_q}\nReasoning:",
    input_variables=["user_q"],
)


def auto_cot(q: str):
    return (auto_template | llm).invoke({"user_q": q}).content


q3 = "John had 12 cookies, gave 5 to friends, then bought 3 more. How many now?"
print(auto_cot(q3))

To find out how many cookies John has now, we start with the number of cookies he initially had, subtract the number he gave away, and then add the number he bought. 

1. Start with 12 cookies.
2. Subtract the 5 cookies he gave to friends: 12 - 5 = 7 cookies.
3. Add the 3 cookies he bought: 7 + 3 = 10 cookies.

Final answer: 10


In [16]:
print(auto_template.invoke({"user_q": "test"}).text)

Question: A farmer has 120 apples. He wants to pack them into boxes, with each box holding 8 apples. How many boxes can he fill completely?
Reasoning: To find out how many boxes the farmer can fill, we divide the total number of apples by the number of apples per box. So, 120 apples ÷ 8 apples/box = 15 boxes.
Final answer: 15


Question: A school has 250 students. If 60% of the students are girls, how many boys are there in the school?
Reasoning: First, we calculate the number of girls by finding 60% of 250. This is 0.60 × 250 = 150 girls. To find the number of boys, we subtract the number of girls from the total number of students: 250 - 150 = 100 boys.
Final answer: 100


Question: A car travels 60 miles per hour. How far will it travel in 3.5 hours?
Reasoning: To find the distance traveled, we use the formula: distance = speed × time. Here, the speed is 60 miles/hour and the time is 3.5 hours. So, distance = 60 × 3.5 = 210 miles.
Final answer: 210


Question: test
Reasoning:
